# Convert taxi zone shapefile to parquet to load into athena
- GTP 1-3-2026
- source: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
- save locally into data/taxi_zones
- need to use python 3.10 for this - use 
``conda env create -f environment.yml`` to create env necessary

In [1]:
import geopandas as gpd
from pyproj import CRS
import boto3
from pathlib import Path

In [2]:
# Test connection
s3 = boto3.client('s3')

# List your S3 buckets
response = s3.list_buckets()
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  amazon-sagemaker-296646497193-us-east-2-53wwjlkhfv8ox5
  geoff-analytics-nyc-tlc-data
  gtp-tlc-data
  tlc-taxi-zones


In [3]:
tlc_gdf = gpd.read_file('../data/taxi_zones/taxi_zones.shp', crs={'EPSG': 2263})

/Users/geoffperrin/miniconda3/envs/tlc/lib/python3.10/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver ESRI Shapefile does not support open option CRS
  return ogr_read(


In [4]:
tlc_gdf['geometry_wkt'] = tlc_gdf['geometry'].apply(lambda x: x.wkt)

# 2. Save as Parquet
tlc_gdf.to_parquet('../data/taxi_zones/taxi_zones.parquet', index=False)

In [5]:
# 3. Upload to S3
s3 = boto3.client('s3')
s3.upload_file('../data/taxi_zones/taxi_zones.parquet', 'tlc-taxi-zones', 'taxi_zones.parquet')

print("Shapefile loaded to S3! Now create the Athena table using the SQL above.")

Shapefile loaded to S3! Now create the Athena table using the SQL above.


### SQL to use in athena:
`` CREATE EXTERNAL TABLE `taxi_zones` (

  OBJECTID int,
  Shape_Leng double,
  Shape_Area double,
  zone string,
  LocationID int,
  borough string,
  geometry string,
  geometry_wkt string
)
STORED AS PARQUET
LOCATION 's3://tlc-taxi-zones/'
TBLPROPERTIES ('parquet.compression'='SNAPPY'); ``